In [110]:
import pandas as pd
import os
import numpy as np
from ensure import ensure_annotations
import re
import warnings
warnings.filterwarnings('ignore')

In [111]:
print(os.getcwd())
print(os.chdir(r'd:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem'))
print(os.getcwd())

d:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem
None
d:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem


In [112]:
df = pd.read_csv(r'data\raw_data\apple_mobile_new_data.csv',index_col=None)

In [113]:
df.drop_duplicates(inplace=True,ignore_index=True)

## Battery

In [114]:
df = df.drop(labels=['battery'],axis=1)

## Storage 

In [115]:
def extract_storage(row):
    storage = row.split()[1]
    return storage
df['storage'] = df['storage_ram'].apply(func=extract_storage)

In [116]:
import warnings
warnings.filterwarnings('ignore')
count = 0
for i in range(0,len(df['storage'])):
    df['storage'][i]= df['storage'].iloc[i].replace('Storage','')
    count+=1
print(f'looped till {count} index')

looped till 424 index


In [117]:
df['storage'] = df['storage'].replace('1','1024')

In [118]:
df['storage'] = df['storage'].astype('Int16')

## Operating System and Processors


In [119]:
def extract_processor(text):
    pattern = r'.*Processor'
    match = re.findall(pattern, text)
    if match:
        return match[0]
    else:
        return text


In [120]:
df['processors'] = df['os_processor'].apply(func=extract_processor)

## IOS Version

In [121]:
def ios_version(text_):
    pattern=r'^\d+[Processor]|\d+[.Processor]'
    found = re.findall(pattern,text_)
    if found:
        return found[0]
    else:
        return text_

In [122]:
df['ios_version'] = df['processors'].apply(ios_version)

In [123]:
df.reset_index(inplace=True)
for Index,i in enumerate(df['ios_version']):
    # print(i)
    i = str(i).replace('P','')
    i = str(i).replace('.','')
    df['ios_version'][Index] = i

In [124]:
non_numeric = [i for i in df['ios_version'] if len(i)> 2]
non_numeric

['Operating SystemiOS 7Operating FrequencyGSM - 850, 900, 1800, 1900; UMTS - 2100',
 'Operating SystemiOS 7Operating FrequencyGSM - 850, 900, 1800, 1900; UMTS - 2100']

In [125]:
df[df['ios_version'] == 'Operating SystemiOS 7Operating FrequencyGSM - 850, 900, 1800, 1900; UMTS - 2100'] # Just Droping

,index,name,ratings,price,imgURL,storage_ram,os_processor,camera,display,network,storage,processors,ios_version
317,317,"APPLE IPhone 4 (Black, 8 GB)",4.5,"₹22,900",https://rukminim2.flixcart.com/image/312/312/m...,Internal Storage8 GBPhone Book MemoryYesCall L...,Operating SystemiOS 7Operating FrequencyGSM - ...,Primary Camera AvailableYesPrimary Camera5MP R...,Display Size8.89 cm (3.5 inch)Resolution640 x ...,Network Type3GSupported NetworksGSMInternet Co...,8,Operating SystemiOS 7Operating FrequencyGSM - ...,Operating SystemiOS 7Operating FrequencyGSM - ...
324,324,"APPLE IPhone 4 (White, 8 GB)",4.6,"₹13,999",https://rukminim2.flixcart.com/image/312/312/m...,Internal Storage8 GBPhone Book MemoryYesCall L...,Operating SystemiOS 7Operating FrequencyGSM - ...,Primary Camera AvailableYesPrimary Camera5MP R...,Display Size8.89 cm (3.5 inch)Resolution640 x ...,Network Type3GSupported NetworksGSMInternet Co...,8,Operating SystemiOS 7Operating FrequencyGSM - ...,Operating SystemiOS 7Operating FrequencyGSM - ...


In [126]:
df = df.drop(df[df['ios_version'] == 'Operating SystemiOS 7Operating FrequencyGSM - 850, 900, 1800, 1900; UMTS - 2100'].index)

In [127]:
df['ios_version'] = df['ios_version'].astype('Int16')

## Processors

In [128]:
def processor(text):
    pattern=r'(?<=Type).*'
    search = re.search(pattern,text)
    if search != None:
        return ' '.join(re.findall(pattern,text))
    else:
        return 'not montioned'

In [129]:
df['processor'] = df['processors'].apply(processor)
df = df.drop('processors',axis=1)
df = df.drop('index',axis=1)


## Price

In [130]:
df.reset_index(inplace=True)
for Index,i in enumerate(df['price']):
    i = str(i).replace('₹','')
    i = str(i).replace(',','')
    df['price'][Index] = i

In [131]:
df['price'] = df['price'].astype('Int16')

In [132]:
df['price'] = df['price'].abs() # this is to remove the negative values from price feature

## Remove columns
1) storage_ram
2) os_processor


In [133]:
df = df.drop(columns=['os_processor','storage_ram'])

## Processor

In [134]:
def processor_extract_dimensity(text):
    splitted_sentence = text.split()

    if 'Mediatek' and 'Dimensity' in splitted_sentence:
        pattern = r'(\d+)+[Processor]'
        string = ' '.join(splitted_sentence)
        match = re.search(pattern,string)
        if match == None:
            if 'Dimensity' == str(splitted_sentence[1]):
                return 'Dimensity'+str(splitted_sentence[2])
            else:
                return 'Dimensity'+str(splitted_sentence[1])
        else:
            find = re.findall(pattern,string)
            return 'Dimensity'+ ''.join(find)
    else:
        return text  


In [135]:
df['processor'] = df['processor'].apply(processor_extract_dimensity)

In [136]:
def processor_extract_ios(text):
    splitted_sentence = text.split()
    pattern = r'^A\d*'
    search = re.search(pattern,' '.join(splitted_sentence))
    if search != None:
        match =re.findall(pattern,' '.join(splitted_sentence))
        return ' '.join(match)
    else:
        return 'otherProcessor'


In [137]:
df['processor'] = df['processor'].apply(processor_extract_ios)

## Network
I don't need in my project I will just drop.

In [138]:
df = df.drop(labels=['network'],axis=1)

## Camera

In [139]:

def camera_extract(row):
    if isinstance(row, str):  # Check if row is a string
        row_splitted = row.split()
        pattern = r'\d+MP'
        words = ' '.join(row_splitted)
        find_cam = re.findall(pattern, words)
        return ' '.join(find_cam)
    else:
        return np.nan


In [140]:
df['camera'] = df['camera'].apply(camera_extract)
# df = df.drop('index',axis=1)

In [141]:
def camera(txt):
    lst = []
    splitted = txt.split()
    for i in splitted:
        make_int = i.replace('MP','')
        lst.append(int(make_int))
    return max(lst)


In [142]:
df['camera'] = df['camera'].apply(camera)

## display

In [143]:
def display_extract(row):
    split_row = row.split()
    if 'LCD' in split_row:
        if 'IPS' in split_row and 'LCD' in split_row:
            return 'IPS LCD'
        return 'LCD'
    
    if 'OLED' in split_row:
        return 'OLED'
    
    if 'TFT' in split_row:
        return 'TFT'
    
    if 'LED' in split_row:
        return 'LED'

    # if 'Adreno' in split_row:
    #     return 'Adreno'
    if 'AMOLED' in split_row:
        if 'HD+' in split_row and 'AMOLED' in split_row:
            return 'HD+ AMOLED'
        return 'AMOLED'
    
    return 'other_types'


In [144]:
df['display'] = df['display'].apply(display_extract)

In [145]:
df

,index,name,ratings,price,imgURL,camera,display,storage,ios_version,processor
0,0,"APPLE iPhone 13 (Blue, 128 GB)",4.7,3537,https://rukminim2.flixcart.com/image/312/312/k...,63,OLED,128,15,A15
1,1,"APPLE iPhone 13 (Midnight, 128 GB)",4.7,3537,https://rukminim2.flixcart.com/image/312/312/k...,63,OLED,128,15,A15
2,2,"APPLE iPhone 13 (Starlight, 128 GB)",4.7,3537,https://rukminim2.flixcart.com/image/312/312/k...,63,OLED,128,15,A15
3,3,"APPLE iPhone 13 (Pink, 128 GB)",4.7,3537,https://rukminim2.flixcart.com/image/312/312/k...,63,OLED,128,15,A15
4,4,"APPLE iPhone 14 (Midnight, 128 GB)",4.6,4463,https://rukminim2.flixcart.com/image/312/312/x...,63,other_types,128,16,A15
...,...,...,...,...,...,...,...,...,...,...
417,419,"APPLE iPhone XR (Yellow, 128 GB)",4.6,23837,https://rukminim2.flixcart.com/image/312/312/k...,63,IPS LCD,128,14,otherProcessor
418,420,"APPLE iPhone 11 Pro (Midnight Green, 256 GB)",4.6,9772,https://rukminim2.flixcart.com/image/312/312/k...,64,other_types,256,13,otherProcessor
419,421,"APPLE iPhone X (Silver, 64 GB)",4.6,26364,https://rukminim2.flixcart.com/image/312/312/j...,63,OLED,64,11,otherProcessor
420,422,"APPLE iPhone X (Space Gray, 256 GB)",4.6,24172,https://rukminim2.flixcart.com/image/312/312/j...,63,OLED,256,11,otherProcessor


In [147]:
df.drop('index',axis=1,inplace=True)

In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         422 non-null    object 
 1   ratings      422 non-null    float64
 2   price        422 non-null    Int16  
 3   imgURL       422 non-null    object 
 4   camera       422 non-null    int64  
 5   display      422 non-null    object 
 6   storage      422 non-null    Int16  
 7   ios_version  422 non-null    Int16  
 8   processor    422 non-null    object 
dtypes: Int16(3), float64(1), int64(1), object(4)
memory usage: 23.6+ KB


In [149]:
for i in df.columns:
    if df[i].dtype == 'int64':
        df[i] = df[i].astype('int16')
    elif df[i].dtype == 'float64':
        df[i] = df[i].astype('float16')

    else:
        pass

In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         422 non-null    object 
 1   ratings      422 non-null    float16
 2   price        422 non-null    Int16  
 3   imgURL       422 non-null    object 
 4   camera       422 non-null    int16  
 5   display      422 non-null    object 
 6   storage      422 non-null    Int16  
 7   ios_version  422 non-null    Int16  
 8   processor    422 non-null    object 
dtypes: Int16(3), float16(1), int16(1), object(4)
memory usage: 18.7+ KB


In [ ]:
df.to_csv(path_or_buf='data\cleaned_dataset/apple_cleaned_data.csv',index=False)